In [6]:
from dotenv import load_dotenv


load_dotenv()

True

# Querying models directly

In [7]:
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI()

hello_message = HumanMessage(content="hello!")

messages = [
    SystemMessage(content="You are a helpful assistant"),
    hello_message,
]


response = model.invoke(messages)

print("printing response: {}".format(response.content))

printing response: Hello! How can I assist you today?


# Prompt templates

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

messages = prompt_template.invoke({"language": "German", "text": "thanks!"})

response = model.invoke(messages)

print("printing response: {}".format(response.content))

printing response: Danke!


# ReAct agent

In [12]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="openai:gpt-4o",  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

# get last message
last_message = result["messages"][-1]

#print to console

print(last_message.content)

The weather in San Francisco is sunny. Enjoy the sunshine!


# LanGraph

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = init_chat_model("gpt-4o-mini", model_provider="openai")


# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


query = "Hi! I'm Joshua."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state


================================== Ai Message ==================================

Hi Joshua! How can I assist you today?
================================== Ai Message ==================================

Your name is Joshua. How can I help you today?
